In [1]:
#IMPORTS
from siamese_network import Comparator, ContrastiveLoss
from SiamesNetworkDataset import SiameseNetworkDataset
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch

In [2]:
#TRAIN DATASET FOR DATALOADER
folder_dataset = datasets.ImageFolder(root="./LFWDataset/archive/lfw-deepfunneled/lfw-deepfunneled")

transformation = transforms.Compose([transforms.Resize((100,100)),
                                    transforms.ToTensor(),
                                    transforms.CenterCrop(50)])
#size of images should be 50x50
lfw_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset, transforms=transformation)

In [3]:
#DATALOADER
train_dataloader = DataLoader(lfw_dataset,
                        shuffle=True,
                        batch_size=64)

In [4]:
#MODEL AND OPTIMIZER
model = Comparator()
criterion = ContrastiveLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005 )

In [5]:
#TRAINING MODEL
num_epochs = 100
print(f'Dataset length: {lfw_dataset.__len__()}')
for epoch in range(num_epochs):
    for i, (img0, img1, label) in enumerate(train_dataloader, 0):
        
        optimizer.zero_grad()
        output1, output2 = model(img0, img1)
        loss_contrastive = criterion(output1, output2, label)
        loss_contrastive.backward()
        optimizer.step()

    if epoch % 10 == 0 :
        print(f"Epoch number {epoch}\n Current loss {loss_contrastive.item()}\n")
    torch.save(model.state_dict(), f'comparator{epoch}.pth') #SAVE WEIGHTS EACH EPOCH